# Graph drawing algorithm A
This note book provides a reference case for _Algorithm A_ for the drawing of hierarchy graphs.
The hierarchy is composed of 7 nodes (including the root node) and is 2-depth symmetrical tree.

In [1]:
import numpy as np
import cvxpy as cp

## Base matrices of the optimization problem
These matrices are written in the real coordinate system. The vertical locations along axis _y_ are stored in vector $\mathbf{Y}$:
$$
\mathbf{Y} = \left< y_{2,1} y_{2,2} y_{3,1} y_{3,2} y_{3,3} y_{3,4} \right>
$$

**Barycentric** relations between parent and child nodes:
$$
[A]\cdot\mathbf{Y} = \mathbb0_3
$$

In [2]:
mat_a = np.array([
    [1, 1, 0, 0, 0, 0],
    [2, 0, -1, -1, 0, 0],
    [0, 2, 0, 0, -1, -1]
])
print("mat_a = \n", mat_a)

mat_a = 
 [[ 1  1  0  0  0  0]
 [ 2  0 -1 -1  0  0]
 [ 0  2  0  0 -1 -1]]


**Minimal distance** relations:
$$
[G]\cdot \mathbf{Y}\ge d_\mathrm{min} \mathbb{1}_4
$$

In [3]:
mat_g = np.array([
    [-1, 1, 0, 0, 0, 0],
    [0, 0, -1, 1, 0, 0],
    [0, 0, 0, -1, 1, 0],
    [0, 0, 0, 0, -1, 1]
])
d_min = 30
u, s, vh = np.linalg.svd(mat_g)
display(mat_g)
with np.printoptions(suppress=True, precision=6):
    print("u = \n", u)
    print("vh = \n", vh)
    print("s = \n", s)

array([[-1,  1,  0,  0,  0,  0],
       [ 0,  0, -1,  1,  0,  0],
       [ 0,  0,  0, -1,  1,  0],
       [ 0,  0,  0,  0, -1,  1]])

u = 
 [[ 0.        1.        0.        0.      ]
 [ 0.5       0.        0.707107  0.5     ]
 [-0.707107  0.        0.        0.707107]
 [ 0.5       0.       -0.707107  0.5     ]]
vh = 
 [[-0.        0.       -0.270598  0.653281 -0.653281  0.270598]
 [-0.707107  0.707107  0.        0.        0.        0.      ]
 [-0.       -0.       -0.5       0.5       0.5      -0.5     ]
 [ 0.        0.       -0.653281 -0.270598  0.270598  0.653281]
 [ 0.533989  0.533989  0.327762  0.327762  0.327762  0.327762]
 [-0.463525 -0.463525  0.377588  0.377588  0.377588  0.377588]]
s = 
 [1.847759 1.414214 1.414214 0.765367]


The **cost function** to reduce:

In [4]:
lform_c = np.array([-1, 1, -1, 0, 0, 1])
display(lform_c)

array([-1,  1, -1,  0,  0,  1])

**Transformation** to unknown vector $\mathbf{X}$ which verifies:
$$
[G]\cdot \mathbf{Y}\ge d_\mathrm{min} \mathbb{1}_4 \iff \mathbf{X} \ge \mathbb0_4
$$
with
$$
\mathbf{X} = \left<\begin{matrix}
y_{2,1} & y_{2,2}-y_{2,1}-d_\mathrm{min} & y_{3,1} & y_{3,2}-y_{3,1}-d_\mathrm{min} & y_{3,3}-y_{3,2}-d_\mathrm{min} & y_{3,4}-y_{3,3}-d_\mathrm{min}
\end{matrix}\right>
$$
which writes:
$$
\mathbf{X} = [P]\cdot\mathbf{Y}-\mathbf{b}
$$
The inverse relationship being:
$$
\mathbf{Y} = [P]^{-1}\cdot\left(\mathbf{X}+\mathbf{b}\right)
$$
Remark: matrix $[P]$ is a unipotent [matrix](https://en.wikipedia.org/wiki/Unipotent) matrix.

In [5]:
mat_p = np.array([
    [1, 0, 0, 0, 0, 0],
    [-1, 1, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0],
    [0, 0, -1, 1, 0, 0],
    [0, 0, 0, -1, 1, 0],
    [0, 0, 0, 0, -1, 1]
])
vect_b = np.array([0, d_min, 0, d_min, d_min, d_min])
inv_mat_p = np.linalg.inv(mat_p)

with np.printoptions(suppress=True):
    print("mat_p = \n", mat_p)
    print("inv_mat_p = \n", inv_mat_p)
    print("vect_b = \n", vect_b)
s, v = np.linalg.eig(mat_p)
print("The eigen values of mat_a:", s)

mat_p = 
 [[ 1  0  0  0  0  0]
 [-1  1  0  0  0  0]
 [ 0  0  1  0  0  0]
 [ 0  0 -1  1  0  0]
 [ 0  0  0 -1  1  0]
 [ 0  0  0  0 -1  1]]
inv_mat_p = 
 [[1. 0. 0. 0. 0. 0.]
 [1. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 1. 0. 0.]
 [0. 0. 1. 1. 1. 0.]
 [0. 0. 1. 1. 1. 1.]]
vect_b = 
 [ 0 30  0 30 30 30]
The eigen values of mat_a: [1. 1. 1. 1. 1. 1.]


In [6]:
np.array([1, 1, 1, 1]) * d_min - np.dot(mat_g @ inv_mat_p, vect_b)
# ev, eb = np.linalg.eig(mat_p)
# display(ev)
# with np.printoptions(suppress=True):
#     display(eb)
# display(np.linalg.cond(mat_p, p=2))
# display(np.invert(mat_p))

array([0., 0., 0., 0.])

In [7]:
# Manually validated
mat_a @ inv_mat_p

array([[ 2.,  1.,  0.,  0.,  0.,  0.],
       [ 2.,  0., -2., -1.,  0.,  0.],
       [ 2.,  2., -2., -2., -2., -1.]])

In [8]:
lform_c @ inv_mat_p

array([0., 1., 0., 1., 1., 1.])

In [9]:
# Manually validated
-np.dot(mat_a @ inv_mat_p, vect_b)

array([-30.,  30.,  90.])

## Solving with Simplex algorithm

In [10]:
# result = linprog(
#     lform_c @ inv_mat_p,
#     A_ub=-mat_g @ inv_mat_p,
#     b_ub=np.zeros(shape=(4,)),
#     A_eq=mat_a @ inv_mat_p,
#     b_eq=-np.dot(mat_a @ inv_mat_p, vect_b),
#     bounds=None,
#     method="simplex",
#     callback=None,
#     options={
#         "maxiter": 5000,
#         "disp": True,
#         "presolve": True,
#         # "tol": 90,
#         "autoscale": False,
#         "rr": True,
#         "bland": False,
#     }
# )
# display(result)

In [11]:
-mat_g @ inv_mat_p @ vect_b + d_min * np.ones(shape=(4,))

array([0., 0., 0., 0.])

## Transformed problem with CVX

In [12]:
x = cp.Variable(shape=(6,), name="X")
constraints = [
    cp.matmul(mat_a @ inv_mat_p, x) == -np.dot(mat_a @ inv_mat_p, vect_b), 
    cp.matmul(mat_g @ inv_mat_p, x) >= 0 #-mat_g @ inv_mat_p @ vect_b + d_min * np.ones(shape=(4,))
]
objective = cp.Minimize(cp.matmul(lform_c @ inv_mat_p, x))
problem = cp.Problem(objective, constraints)
solution = problem.solve()

In [13]:
print(solution)
print(x.value)

29.99999993656985
[-3.00000000e+01  3.00000000e+01 -4.50000000e+01 -1.70470347e-08
  1.09460941e-08 -3.98345087e-08]


In [18]:
with np.printoptions(suppress=True, precision=6):
    print("Solution in y-coordinates = \n", inv_mat_p @ (x.value + vect_b))

Solution in y-coordinates = 
 [-30.  30. -45. -30.  15.  90.]


## Original problem with CVX

In [15]:
x = cp.Variable(shape=(6,), name="X")
constraints = [cp.matmul(mat_a, x) == 0, mat_g @ x >= d_min * np.ones(shape=(4,))]
objective = cp.Minimize(cp.matmul(lform_c, x))
problem = cp.Problem(objective, constraints)
solution = problem.solve()

In [19]:
with np.printoptions(suppress=True, precision=6):
    print("Solution in y-coordinates = \n", x.value)

Solution in y-coordinates = 
 [-30.  30. -45. -15.  15.  45.]
